In [47]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
fresh_name = "../data/0-7/0hox6-0.png"
img = cv2.imread(fresh_name, 0)
cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [48]:
# 2a. Apply median filtering to reduce noise and display the result.
# img = cv2.medianBlur(img, 5)
# cv2.imshow("img", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [49]:
# 2b. Got smearing? consider using an inpainting technique to clean up the image.
mask = (img < 30).astype(np.uint8) * 255
img = cv2.inpaint(img, mask, 3, cv2.INPAINT_TELEA)
cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [50]:
# 3** This tool should only be applied if you suspect the letters are broken into pieces.
# simple heuristic: if the number of contours detected is more than the expected number of letters
# kernel = np.ones((3, 3), np.uint8)
# img = cv2.erode(img, kernel, iterations=2)
# img = cv2.dilate(img, kernel, iterations=2)
# cv2.imshow("img", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [51]:
bounding_boxes = []
threshed_img = img.copy()
ret, thresh = cv2.threshold(threshed_img, 250, 255, cv2.THRESH_BINARY_INV)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

img = cv2.imread(fresh_name)  # Read in color for drawing
for i, c in enumerate(contours):
    parent = hierarchy[0][i][3]
    if parent != -1:  # Skip if it has a parent contour    
        continue
    x, y, w, h = cv2.boundingRect(c)
    bounding_boxes.append((x, y, w, h))
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

In [52]:
cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [53]:
# Tool to sort bounding boxes based on their leftmost x-coordinate
def sort_bounding_boxes(boxes):
    return sorted(boxes, key=lambda box: box[0])
bounding_boxes = sort_bounding_boxes(bounding_boxes)

In [54]:
# Find a way to learn how many colors are in a contour, if more than 1 there's overlapped letters in the same box
img = cv2.imread(fresh_name)  # Read in fresh image without the bounding boxes
focus = bounding_boxes[0]
x, y, w, h = focus
roi = img[y:y + h, x:x + w]
# try and see how many unique colors are in this roi
roi_rgb = roi.reshape(-1, 3)
colors, counts = np.unique(roi_rgb, axis=0, return_counts=True)
print(f"Unique colors: {len(colors)}")
# sort by leftmost pixel
top_color_counts = sorted(zip(counts, colors), key=lambda x: x[0], reverse=True)
# look at top 5 colors
for count, color in top_color_counts[:8]:
    print(f"Color: {color}, Count: {count}")


Unique colors: 70
Color: [255 255 255], Count: 298
Color: [ 91 201 147], Count: 152
Color: [172 195 142], Count: 98
Color: [0 0 0], Count: 29
Color: [160 224 192], Count: 25
Color: [252 254 253], Count: 20
Color: [ 96 202 150], Count: 13
Color: [152 221 187], Count: 13


In [55]:
# uncomment if scikit-learn is not installed
# !{sys.executable} -m pip install scikit-learn

In [56]:
# ignoring near white and near black colors
usable_colors = []
usable_counts = []
for count, color in top_color_counts[:8]:
    if np.linalg.norm(color - 255) > 5 and np.linalg.norm(color) > 5 and count > w * h * 0.01:
        usable_colors.append(color)
        usable_counts.append(count)

if len(usable_counts) == 0:
    print("No usable colors found.")
    exit(0)
elif(len(usable_counts) == 1):
    print("Only one usable color found. Estimated number of colors (k): 1")
    exit(0)

print(f"Usable colors: {len(usable_colors)}")
print("Usable color counts:", usable_counts)
# the arrays should already be sorted by count
# find sharp dropoff
ratios = usable_counts[:-1] / np.array(usable_counts[1:]) + 1e-5  # avoid div by 0
est_k = np.argmax(ratios) + 1

print(f"Estimated number of colors (k): {est_k}")

Usable colors: 6
Usable color counts: [np.int64(152), np.int64(98), np.int64(25), np.int64(20), np.int64(13), np.int64(13)]
Estimated number of colors (k): 2
